In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix

# Define directories
train_dir = "./data and predictions/test"
test_dir = './data and predictions/train'
val_dir = './data and predictions/valid'

# Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess data
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='binary')
validation_generator = test_datagen.flow_from_directory(val_dir,
                                                        target_size=(150, 150),
                                                        batch_size=32,
                                                        class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(150, 150),
                                                  batch_size=32,
                                                  class_mode='binary')


Found 601 images belonging to 2 classes.
Found 1165 images belonging to 2 classes.
Found 12229 images belonging to 2 classes.


In [2]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


c:\Users\TUSHAR SAIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [3]:
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // 32,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples // 32,
                    epochs=10)


Epoch 1/10


c:\Users\TUSHAR SAIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


18/18 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.6198 - loss: 0.9739 - val_accuracy: 0.7396 - val_loss: 0.6049
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7600 - loss: 0.3128 - val_accuracy: 0.6923 - val_loss: 0.3025
Epoch 3/10


c:\Users\TUSHAR SAIN\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


18/18 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.6855 - loss: 0.6058 - val_accuracy: 0.8238 - val_loss: 0.3875
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7812 - loss: 0.2459 - val_accuracy: 0.7692 - val_loss: 0.2075
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.7523 - loss: 0.4874 - val_accuracy: 0.8950 - val_loss: 0.2611
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8750 - loss: 0.1585 - val_accuracy: 0.7692 - val_loss: 0.2390
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.8538 - loss: 0.3352 - val_accuracy: 0.8828 - val_loss: 0.2780
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7812 - loss: 0.2555 - val_accuracy: 0.9231 - val_loss: 0.1313
Epoch 9/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.8279 - loss: 0.4126 - val_accuracy: 0.9036 - val_loss: 0.2451
Epoch 10/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9375 - loss: 0.1651 - val_accuracy: 1.0000 - val_loss: 0.0445


In [5]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // 32)
print(f'Test accuracy: {test_acc}')

# Generate classification report and confusion matrix
test_generator.reset()
Y_pred = model.predict(test_generator, test_generator.samples // 32+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['Normal', 'Pneumonia']
print(classification_report( test_generator.classes,y_pred, target_names=target_names))


382/382 ━━━━━━━━━━━━━━━━━━━━ 83s 217ms/step - accuracy: 0.8893 - loss: 0.2757
Test accuracy: 0.8871891498565674
383/383 ━━━━━━━━━━━━━━━━━━━━ 79s 205ms/step
Confusion Matrix
[[3312    0]
 [8917    0]]
Classification Report
              precision    recall  f1-score   support

      Normal       0.27      1.00      0.43      3312
   Pneumonia       0.00      0.00      0.00      8917

    accuracy                           0.27     12229
   macro avg       0.14      0.50      0.21     12229
weighted avg       0.07      0.27      0.12     12229



c:\Users\TUSHAR SAIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\TUSHAR SAIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\TUSHAR SAIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

In [7]:
model.save('pneumonia_model.h5')

